In [19]:
from typing import Tuple
import hydra
from hydra.utils import instantiate
import logging
from omegaconf import DictConfig
from pathlib import Path
import pytorch_lightning as pl
from torch.utils.data import DataLoader

from navsim.planning.training.dataset import CacheOnlyDataset, Dataset
from navsim.planning.training.agent_lightning_module import AgentLightningModule
from navsim.common.dataloader import SceneLoader
from navsim.common.dataclasses import SceneFilter
from navsim.agents.abstract_agent import AbstractAgent

In [20]:

def build_datasets(cfg: DictConfig, agent: AbstractAgent) -> Tuple[Dataset, Dataset]:
    train_scene_filter: SceneFilter = instantiate(cfg.scene_filter)
    if train_scene_filter.log_names is not None:
        train_scene_filter.log_names = [l for l in train_scene_filter.log_names if l in cfg.train_logs]
    else:
        train_scene_filter.log_names = cfg.train_logs

    val_scene_filter: SceneFilter = instantiate(cfg.scene_filter)
    if val_scene_filter.log_names is not None:
        val_scene_filter.log_names = [l for l in val_scene_filter.log_names if l in cfg.val_logs]
    else:
        val_scene_filter.log_names = cfg.val_logs

    data_path = Path(cfg.navsim_log_path)
    sensor_blobs_path = Path(cfg.sensor_blobs_path)

    train_scene_loader = SceneLoader(
        sensor_blobs_path=sensor_blobs_path,
        data_path=data_path,
        scene_filter=train_scene_filter,
        sensor_config=agent.get_sensor_config(),
    )

    val_scene_loader = SceneLoader(
        sensor_blobs_path=sensor_blobs_path,
        data_path=data_path,
        scene_filter=val_scene_filter,
        sensor_config=agent.get_sensor_config(),
    )

    train_data = Dataset(
        scene_loader=train_scene_loader,
        feature_builders=agent.get_feature_builders(),
        target_builders=agent.get_target_builders(),
        cache_path=cfg.cache_path,
        force_cache_computation=cfg.force_cache_computation,
    )

    val_data = Dataset(
        scene_loader=val_scene_loader,
        feature_builders=agent.get_feature_builders(),
        target_builders=agent.get_target_builders(),
        cache_path=cfg.cache_path,
        force_cache_computation=cfg.force_cache_computation,
    )

    return train_data, val_data

In [21]:
from omegaconf import OmegaConf
cfg = OmegaConf.load("/root/navsim/cfg.yaml")

In [22]:
pl.seed_everything(0, workers=True)

Seed set to 0


0

In [23]:
agent: AbstractAgent = instantiate(cfg.agent)

In [24]:
lightning_module = AgentLightningModule(
        agent=agent,
    )

In [25]:
train_data, val_data = build_datasets(cfg, agent)

KeyboardInterrupt: 

In [ ]:
train_dataloader = DataLoader(train_data, **cfg.dataloader.params, shuffle=True)

In [ ]:
val_dataloader = DataLoader(val_data, **cfg.dataloader.params, shuffle=False)

In [ ]:
data=train_data[0]

In [ ]:
data1=train_data[1]

In [ ]:
data

({'camera_feature': tensor([[[0.2863, 0.2510, 0.2667,  ..., 0.2471, 0.2510, 0.2471],
           [0.2706, 0.2588, 0.2549,  ..., 0.2471, 0.2471, 0.2471],
           [0.2784, 0.2667, 0.2667,  ..., 0.2471, 0.2471, 0.2471],
           ...,
           [0.6196, 0.6118, 0.6196,  ..., 0.2549, 0.2549, 0.2510],
           [0.6235, 0.6314, 0.6471,  ..., 0.2471, 0.2510, 0.2471],
           [0.6196, 0.6275, 0.6314,  ..., 0.2431, 0.2431, 0.2392]],
  
          [[0.2863, 0.2471, 0.2588,  ..., 0.2941, 0.2980, 0.2941],
           [0.2824, 0.2353, 0.2353,  ..., 0.2941, 0.2941, 0.2941],
           [0.2902, 0.2471, 0.2549,  ..., 0.2941, 0.2941, 0.2941],
           ...,
           [0.6588, 0.6471, 0.6471,  ..., 0.2667, 0.2667, 0.2627],
           [0.6588, 0.6588, 0.6706,  ..., 0.2588, 0.2627, 0.2588],
           [0.6431, 0.6471, 0.6471,  ..., 0.2627, 0.2627, 0.2588]],
  
          [[0.2627, 0.2353, 0.2706,  ..., 0.3490, 0.3529, 0.3490],
           [0.2706, 0.2275, 0.2431,  ..., 0.3412, 0.3412, 0.3412],
    

In [ ]:
data1[1]["trajectory"]

tensor([[ 3.6928e+00,  1.3640e-02, -1.6618e-03],
        [ 7.4971e+00,  2.2371e-02, -2.2717e-03],
        [ 1.1372e+01, -9.8581e-03, -2.1750e-03],
        [ 1.5332e+01, -3.3419e-02, -2.8064e-03],
        [ 1.9347e+01, -5.1332e-02, -2.8773e-03],
        [ 2.3422e+01, -6.1071e-02, -2.7041e-03],
        [ 2.7508e+01, -8.1003e-02, -2.8067e-03],
        [ 3.1510e+01, -9.8640e-02, -2.1192e-03]], dtype=torch.float64)

In [ ]:
data[1]["trajectory"]

tensor([[ 1.8449, -0.0883, -0.1092],
        [ 3.6463, -0.4023, -0.2398],
        [ 5.3686, -0.9523, -0.3889],
        [ 7.0978, -1.7980, -0.5564],
        [ 8.7931, -3.0453, -0.7379],
        [10.3725, -4.7149, -0.9012],
        [11.8347, -6.7531, -0.9963],
        [13.2534, -9.0238, -1.0544]], dtype=torch.float64)

In [ ]:
data[0]["camera_feature"].shape

torch.Size([3, 256, 1024])

In [ ]:
data[0]["lidar_feature"].shape

torch.Size([1, 256, 256])

In [ ]:
data[0]["status_feature"].shape

torch.Size([8])

In [ ]:
data[1]["agent_states"].shape

torch.Size([30, 5])